In [2]:
%load_ext autoreload
%autoreload 2

from IPython.display import display

import pandas as pd

import CBOEFileParser
import VerticalSpreads
import CBOE_Downloader
import POP

In [16]:
%%capture


underlying_symbol = "AAPL"
risk_free_rate = 2.98

cboe_file = CBOE_Downloader.CBOE_Downloader(underlying_symbol).download().save_download().get_filename()
option_chains = CBOEFileParser.CBOEFileParser() \
                    .parse_cboe_file(cboe_file) \
                    .remove_options_after(days=60) \
                    .remove_options_before(days=30) \
                    .set_exchange() \
                    .add_greeks(risk_free_rate=risk_free_rate)

In [17]:
pop_calculator = POP.POP().get_ohlc_data(underlying_symbol)

In [18]:
vertical_spreads = VerticalSpreads.VerticalSpreads() \
                                    .build_spreads(option_chains) \
                                    .risk_limit(200) \
                                    .itm_spreads(option_chains.underlying_spot) \
                                    .add_pop(pop_calculator) \
                                    .sky_view_spreads()
print(option_chains.underlying_symbol)
print("Current Spot: {}".format(option_chains.underlying_spot))

pd.options.display.max_columns = None
for expiration, bull_calls, bull_puts, bear_calls, bear_puts in vertical_spreads.items(pretty_df=True):
    
    print("Expiration: {}".format(expiration))

    print("Bull Puts:")
    if isinstance(bull_puts, pd.DataFrame):
        display(bull_puts.sort_values(by=['Break Even'], ascending=True).head(20))

#     print("Bear Calls:")
#     if isinstance(bear_calls, pd.DataFrame):
#         display(bear_calls.sort_values(by=['Delta'], ascending=True).head(20))

AAPL
Current Spot: 191.52
Expiration: 2018-09-07 16:00:00
Bull Puts:


,Strike Display,Max Risk,Max Profit,Break Even,POP,Delta,Gamma,Theta,Vega
196,180.0 / 182.5,-1.99,0.51,181.99,0.937,0.0001,-0.0000,-0.0001,-0.0003
216,182.5 / 185.0,-1.95,0.55,184.45,0.885,0.0002,-0.0000,-0.0001,-0.0004
235,185.0 / 187.5,-1.80,0.70,186.80,0.825,0.0004,-0.0001,-0.0003,-0.0007


Expiration: 2018-09-21 16:00:00
Bull Puts:


,Strike Display,Max Risk,Max Profit,Break Even,POP,Delta,Gamma,Theta,Vega


Expiration: 2018-08-31 16:00:00
Bull Puts:


,Strike Display,Max Risk,Max Profit,Break Even,POP,Delta,Gamma,Theta,Vega
286,180.0 / 182.5,-1.97,0.53,181.97,0.921,0.0003,-0.0001,-0.0002,-0.0005
306,182.5 / 185.0,-1.92,0.58,184.42,0.877,0.0005,-0.0001,-0.0003,-0.0008
325,185.0 / 187.5,-1.80,0.70,186.80,0.810,0.0009,-0.0001,-0.0007,-0.0015
343,187.5 / 190.0,-1.70,0.80,189.20,0.663,0.0015,-0.0002,-0.0011,-0.0023
